## Day 01

In [23]:
# importing the libaries
import openai
import os 
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import display, Markdown

In [5]:
# load the environment variables from .env file
load_dotenv()

# set the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Testing the OpenAI API with a simple message
# This is a simple test to check if the OpenAI API is working correctly
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's great to hear from you! How can I assist you today?


In [7]:
# Some websites need you to use proper headers when fetching them:
headers = {
    # The User-Agent header is used to identify the client making the request.
    # This helps in mimicking a real browser to avoid being blocked by websites.
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Initialize a Website object from the given URL using the BeautifulSoup library.
        
        Args:
            url (str): The URL of the website to fetch and parse.
        """
        self.url = url  # Store the URL of the website.

        # Send an HTTP GET request to the URL with the specified headers.
        response = requests.get(url, headers=headers)

        # Parse the HTML content of the response using BeautifulSoup.
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the title of the webpage, if available. If not, set it to "No title found".
        self.title = soup.title.string if soup.title else "No title found"

        # Remove irrelevant elements like <script>, <style>, <img>, and <input> from the body of the webpage.
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        # Extract the visible text from the body of the webpage, separating lines with a newline character.
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Let's try one out. Change the website and add print statements to follow along.
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

### Types of prompts¶
You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

A system prompt that tells them what task they are performing and what tone they should use

A user prompt -- the conversation starter that they should reply to

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages
The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

```py
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [16]:
messages = [
    {'role': 'system', 'content': 'You are a snarky assistant'},
    {'role': 'user', 'content': 'What is 2+2?'}
]

In [17]:
# To give you a preview -- calling OpenAI with system and user messages:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Well, if you’re asking me, it’s clearly 4. But let’s be real, in some alternate universe, it could be 5. Who knows? Math can be so mysterious!


### And now let's build useful messages for GPT-4o-mini, using a function

In [18]:
def message_for(website):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt_for(website)}
    ]

In [19]:
message_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [20]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_for(website)
    )

    return response.choices[0].message.content

In [24]:
display(Markdown(summarize("https://edwarddonner.com")))

# Summary of Edward Donner's Website

Edward Donner's personal website showcases his interests and professional endeavors. He is a technology enthusiast, particularly in the field of artificial intelligence and large language models (LLMs). As the co-founder and CTO of Nebula.io, he focuses on leveraging AI to enhance talent discovery and management, providing tools for recruiters. Previously, he founded the startup untapt, which was acquired in 2021.

In addition to his tech work, Ed has a passion for DJing, electronic music production, and engaging with the tech community through platforms like Hacker News.

## Recent Posts
- **April 21, 2025**: Announcement of "The Complete Agentic AI Engineering Course".
- **January 23, 2025**: Information about an "LLM Workshop – Hands-on with Agents", including available resources.
- **December 21, 2024**: Welcome message to "SuperDataScientists".
- **November 13, 2024**: Resources shared for "Mastering AI and LLM Engineering".

The website also encourages visitors to connect with Ed through various social platforms and a newsletter subscription.

In [25]:
display(Markdown(summarize("https://cnn.com")))

# CNN Website Summary

CNN provides a comprehensive platform for breaking news, latest updates, and videos across various categories including US and world news, politics, business, health, entertainment, sports, and more. The website emphasizes real-time reporting and analysis on significant global events, including:

## Recent News Highlights
- **Aid Ship Incident**: An aid ship bound for Gaza catches fire after an alleged Israeli drone attack.
- **Trump's Tariffs**: Recent tariffs imposed by Trump are poised to affect millions of Americans, especially regarding shipping exemptions and pricing on goods.
- **Ukraine-Russia War**: Continuous updates reflect the ongoing conflict, including a recent minerals deal signed by the US with Kyiv amid projections that the war will not end soon.
- **Cultural Events**: The world mourns the passing of Inah Canabarro Lucas, the world’s oldest person, who lived to 116 years old.

## Key Features
- **Live Updates**: The site includes a section for live updates on breaking news.
- **Multimedia Content**: Users can access videos and articles analyzing current events, stock market trends, and significant political developments.
- **Global Perspectives**: Content explores stories from various regions, providing insight into international affairs, cultural shifts, and scientific advancements.

CNN aims to keep its audience informed with the latest developments and offers an array of topics for readers to explore, reinforcing its role as a leading news source.

In [28]:
print(summarize("https://anthropic.com"))

# Website Summary for Anthropic

The Anthropic website primarily focuses on its flagship AI model, Claude, showcasing its capabilities, pricing plans, and various applications. Key sections of the site include:

- **Claude Overview**: Introduction to Claude and its various iterations (like Claude 3.7 Sonnet), highlighting its advanced capabilities and responsible AI design.
- **API**: Resources for developers to build applications with Claude, including API documentation and pricing.
- **Solutions**: Information on how Claude can assist in different domains such as AI agents, coding, and customer support.
- **Research**: Insights into the company's research initiatives concerning AI safety, impacts, and transparency. Notable mentioned topics include the Claude Economic Index and interpretability.
- **Learning Resources**: The Anthropic Academy offers tutorials on utilizing Claude effectively.
- **Responsible AI Development**: An emphasis on creating AI technologies with human benefit i

In [26]:
display(Markdown(summarize("https://anthropic.com")))

# Summary of the Anthropic Website

The Anthropic website primarily focuses on their AI product, Claude, emphasizing safe and responsible AI development. Key highlights include:

## AI Products and Features
- **Claude 3.7 Sonnet**: The latest iteration of their intelligent AI model is presented as the most advanced version available for use.
- **API Opportunities**: Users can build AI-powered applications and experiences using Claude through a comprehensive API.

## Research and Development
- Anthropic prioritizes the long-term well-being of humanity in the development of AI, advocating for responsible technology design.
- **Recent Research Topics**:
  - Tracing thoughts of large language models.
  - The Anthropic Economic Index and societal impacts.
  - Discussions on AI alignment and interpretability.

## News and Announcements
- The site announces **ISO 42001 certification**, marking a commitment to safety and compliance in AI practices.

## Education and Resources
- The **Anthropic Academy** offers learning resources to help users build with Claude effectively.

With a clear focus on advancing AI technologies safely, Anthropic aims to provide tools that prioritize human benefit and address societal concerns related to AI advancements.